# Titanic Machine Learning Example

In [ ]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict

In [ ]:
# Load in the train and test datasets
train = pd.read_csv('/Users/adam//Downloads/titanic/train.csv')
test = pd.read_csv('/Users/adam//Downloads/titanic/test.csv')

# Store our passenger ID for easy access
PassengerId = test['PassengerId']

train.head(3)

## Feature Engineering

As this notebook is designed to be used for classification ML. I have used the titanic dataset and feature engineering from another kaggle competitor so that I can focus on developing the ML pipeline

In [ ]:
full_data = [train, test]

# Some features of my own that I have added in
# Gives the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)

# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Feature engineering steps taken from Sina
# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    
# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)

# Create a New feature CategoricalAge
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
train['CategoricalAge'] = pd.cut(train['Age'], 5)

# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
    
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4

In [ ]:
# Feature selection
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
test  = test.drop(drop_elements, axis = 1)


In [ ]:
train.head(3)

In [ ]:
#colormap = plt.cm.RdBu
#plt.figure(figsize=(14,12))
#plt.title('Pearson Correlation of Features', y=1.05, size=15)
#sns.heatmap(train.astype(float).corr(),linewidths=0.1,vmax=1.0, 
#            square=True, cmap=colormap, linecolor='white', annot=True)

## Model Preparation

In [ ]:
#remove labels from data
labels = np.array(train['Survived'])
train = train.drop('Survived', axis = 1)


#Split training dataset into train and validation sets
train_features, valid_features, train_labels, valid_labels = train_test_split(train, labels, test_size = 0.1, random_state = 42)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Validation Features Shape:', valid_features.shape)
print('Validation Labels Shape:', valid_labels.shape)

# Training the Model

## Make A Function for Classification ML

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
#feature_importance = lambda forest_class: forest_class.feature_importances_
#svm_importance = lambda sup_class: sup_class.coef_[0] #TODO: Review coef_ docs

NO_ESTIMATORS = 500

classifiers = [
    ['Random Forest', RandomForestClassifier(n_jobs = -1 , n_estimators = NO_ESTIMATORS, warm_start = True, max_depth= 6, max_features='sqrt'), 0],
    ['Extra Trees', ExtraTreesClassifier(n_jobs= -1, n_estimators = NO_ESTIMATORS, max_depth = 8, min_samples_leaf = 2), 0],
    ['AdaBoost', AdaBoostClassifier(n_estimators = NO_ESTIMATORS, learning_rate = 0.75), 0],
    ['Gradient Boost', GradientBoostingClassifier(n_estimators = NO_ESTIMATORS, max_depth = 5, min_samples_leaf = 2), 0],
    #GaussianProcessClassifier(1.0 * RBF(1.0))
    ['SVC', SVC(kernel = 'linear', C = 0.025), 1],
    ['XGBoost', xgb.XGBClassifier(n_estimators= NO_ESTIMATORS, max_depth= 4, min_child_weight= 2, gamma=0.9, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', nthread= -1, scale_pos_weight=1), 0]
]

model_type = []
for model in classifiers:
    model_type.append(model[0])

In [ ]:
# Definition of Models Used

m1 = ['Random Forest', RandomForestClassifier(n_jobs = -1 , n_estimators = NO_ESTIMATORS, warm_start = True, max_depth= 6, max_features='sqrt'), 0]
m2 = ['Extra Trees', ExtraTreesClassifier(n_jobs= -1, n_estimators = NO_ESTIMATORS, max_depth = 8, min_samples_leaf = 2), 0]
m3 = ['AdaBoost', AdaBoostClassifier(n_estimators = NO_ESTIMATORS, learning_rate = 0.75), 0]
m4 = ['Gradient Boost', GradientBoostingClassifier(n_estimators = NO_ESTIMATORS, max_depth = 5, min_samples_leaf = 2), 0]
m5 = ['SVC', SVC(kernel = 'linear', C = 0.025), 1]
m6 = ['XGBoost', xgb.XGBClassifier(n_estimators= NO_ESTIMATORS, max_depth= 4, min_child_weight= 2, gamma=0.9, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', nthread= -1, scale_pos_weight=1), 0]


df = pd.DataFrame(columns = [model_type])
df.append(m1)
df

In [ ]:
m1

### Loop That Does The ML

In [ ]:
scoring = ['accuracy', 'precision', 'recall']
accuracy_df = pd.DataFrame(index= scoring)
importance_df = pd.DataFrame(index= train.columns)
predictions = []
CROSS_VALIDATION_NUMBER = 5
##TODO: Add Random Seed Number

for classifier in classifiers:
    print('Currently Modelling Using', classifier[0])
    model = classifier[1].fit(train_features, train_labels)
    predictions.append(model.predict(valid_features))
    errors = abs(predictions - valid_labels)
    
    if classifier[2] == 0:
        importance_df[classifier[0]] = classifier[1].feature_importances_
    if classifier[2] == 1:
        importance_df[classifier[0]] = abs(classifier[1].coef_[0])
        
    for i in scoring:
        xval_score = cross_val_score(model, valid_features, valid_labels, cv=CROSS_VALIDATION_NUMBER, scoring=i)
        accuracy_df.loc[i, classifier[0]] = np.mean(xval_score)
        
    ## ADD a field to store predictions

also look to add/remove features in a brute force approach to see the effect that has on feature importance

## Plotting Feature Importance

In [ ]:
_temp = importance_df
_temp.reset_index(level=0, inplace=True)
_temp = _temp.rename(columns= {"index": "Feature"})
_temp = pd.melt(_temp, id_vars=['Feature'], value_vars=model_type)

import seaborn as sns
sns.factorplot(x='Feature', y='value', hue='variable', data=_temp, kind='bar', aspect = 3)

## Plotting Model Accuracy

In [ ]:
_temp = accuracy_df
_temp.reset_index(level=0, inplace=True)
_temp = _temp.rename(columns= {"index": "Method"})
_temp = pd.melt(_temp, id_vars=['Method'], value_vars=model_type)

sns.factorplot(x='variable', y='value', hue='Method', data=_temp, kind='bar', aspect = 3)

## Ensemble Learning: Voting on Predictions

In order to vote on predictions, I need to have a system where the voting can grab each model again

In [ ]:
len(predictions)

In [ ]:
temp_df = pd.DataFrame


for i in range(0, len(predictions)):
    temp_df.append(predictions[i])
    print(predictions[i])
    
    
    
    


In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

rf= RandomForestClassifier(n_jobs = -1 , n_estimators = NO_ESTIMATORS, warm_start = True, max_depth= 6, max_features='sqrt')
et= ExtraTreesClassifier(n_jobs= -1, n_estimators = NO_ESTIMATORS, max_depth = 8, min_samples_leaf = 2)
ab= AdaBoostClassifier(n_estimators = NO_ESTIMATORS, learning_rate = 0.75)
gb= GradientBoostingClassifier(n_estimators = NO_ESTIMATORS, max_depth = 5, min_samples_leaf = 2)
svc = SVC(kernel = 'linear', C = 0.025)
xg = xgb.XGBClassifier(n_estimators= NO_ESTIMATORS, max_depth= 4, min_child_weight= 2, gamma=0.9, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', nthread= -1, scale_pos_weight=1)


eclf1 = VotingClassifier(estimators=[rf, et], voting='hard')
eclf1 = eclf1.fit(train_features, train_labels)
print(eclf1.predict(train_features))

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
clf1 = LogisticRegression(solver='lbfgs', multi_class='multinomial', random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
y = np.array([1, 1, 1, 2, 2, 2])
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

model_type = []

for i in classifiers:
    model_type.append(i[1])

#eclf1 = VotingClassifier(estimators=[model_type], voting='hard')


eclf1 = eclf1.fit(train_features, train_labels)
print('ECLF1')
print(eclf1.predict(train_features))



In [ ]:

    
model_type